In this notebook we'll explore topic modeling to discover broad themes in a collection of movie summaries.

In [1]:
import nltk
import re
import gensim
from gensim import corpora
import operator

nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import random

random.seed(1)

/Users/dbamman/opt/anaconda3/envs/anlp/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dbamman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def read_stopwords(filename):
    stopwords={}
    with open(filename) as file:
        for line in file:
            stopwords[line.rstrip()]=1
    return stopwords

Since we're running topic modeling on texts with lots of names, we'll add the Jockers list of stopwords (which includes character names) to our stoplist.

In [3]:
stop_words = {k:1 for k in stopwords.words('english')}
stop_words.update(read_stopwords("../data/jockers.stopwords"))
stop_words["'s"]=1
stop_words=list(stop_words.keys())

In [4]:
def filter(word, stopwords):
    
    """ Function to exclude words from a text """
    
    # no stopwords
    if word in stopwords:
        return False
    
    # has to contain at least one letter
    if re.search("[A-Za-z]", word) is not None:
        return True
    
    return False

In [5]:
def read_docs(plotFile, metadataFile, stopwords):
    
    names={}
    box={}
    
    with open(metadataFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            name=cols[2]
            boxoffice=cols[4]
            if len(boxoffice) != 0:
                box[idd]=int(boxoffice)
                names[idd]=name
    
    n=5000
    target_movies={}


    sorted_box = sorted(box.items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_box[:n]:
        target_movies[k]=names[k]
    
    docs=[]
    names=[]
   
    with open(plotFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            text=cols[1]
            
            if idd in target_movies:
                tokens=nltk.word_tokenize(text.lower())
                tokens=[x for x in tokens if filter(x, stopwords)]
                docs.append(tokens)
                name=target_movies[idd]
                names.append(name)
    return docs, names

We'll read in summaries of the 5,000 movies with the highest box office revenues.

In [8]:
metadataFile="../data/movie.metadata.tsv"
plotFile="../data/plot_summaries.txt"
data, doc_names=read_docs(plotFile, metadataFile, stop_words)

We will convert the movie summaries into a bag-of-words representation using gensim's [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) methods.

In [9]:
# Create vocab from data; restrict vocab to only the top 10K terms that show up in at least 5 documents 
# and no more than 50% of all documents

dictionary = corpora.Dictionary(data)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)

In [10]:
# Replace dataset with numeric ids words in vocab (and exclude all other words)
corpus = [dictionary.doc2bow(text) for text in data]

In [11]:
num_topics=20

Now let's run a topic model on this data using gensim's built-in LDA.

In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           passes=10,
                                           alpha='auto')

We can get a sense of what the topics are by printing the top 10 words with highest $P(word \mid topic)$ for each topic

In [14]:
for i in range(num_topics):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model.show_topic(i, topn=10)])))

topic 0:	house find finds body room begins killed goes tries inside
topic 1:	town prison case sheriff judge smith trial murder police court
topic 2:	school tells father time life day mother film friends friend
topic 3:	earth alien planet ship machine space aliens humans crew human
topic 4:	stone kid dj vega kramer pop smoke play party stab
topic 5:	mrs. boone camp kids cat hayes whale butch tooth stark
topic 6:	team game coach football west players player play win national
topic 7:	dr. evil de satan god soul death body father priest
topic 8:	father family mother away find help named tells children return
topic 9:	white doc snow burke jones shark penguins oz pink penguin
topic 10:	car police tells money house goes finds find apartment night
topic 11:	agent bomb bond fbi plane agents kill escape killed president
topic 12:	fight battle army kill robot soldiers city attack men orders
topic 13:	dracula vampire toys count silver werewolf barnes bug toy ferguson
topic 14:	war army soldiers me

Another way of understanding topics is to print out the documents that have the highest topic representation -- i.e., for a given topic $k$, the documents with highest $P(topic=k | document)$.  How much do the documents listed here align with your understanding of the topics?

In [15]:
topic_model=lda_model 

topic_docs=[]
for i in range(num_topics):
    topic_docs.append({})
for doc_id in range(len(corpus)):
    doc_topics=topic_model.get_document_topics(corpus[doc_id])
    for topic_num, topic_prob in doc_topics:
        topic_docs[topic_num][doc_id]=topic_prob

for i in range(num_topics):
    print("%s\n" % ' '.join([term for term, freq in topic_model.show_topic(i, topn=10)]))
    sorted_x = sorted(topic_docs[i].items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x[:5]:
        print("%s\t%.3f\t%s" % (i,v,doc_names[k]))
    print()
    
    

house find finds body room begins killed goes tries inside

0	0.970	The Final Destination
0	0.940	A Nightmare on Elm Street 2: Freddy's Revenge
0	0.906	Friday the 13th
0	0.878	Deadly Friend
0	0.870	A Nightmare on Elm Street

town prison case sheriff judge smith trial murder police court

1	0.940	Needful Things
1	0.897	Without a Clue
1	0.874	Murder in the First
1	0.804	Another Public Enemy
1	0.733	The Shawshank Redemption

school tells father time life day mother film friends friend

2	0.997	Prom
2	0.995	Shopgirl
2	0.995	National Lampoon's Van Wilder
2	0.995	Parenthood
2	0.993	Friends with Kids

earth alien planet ship machine space aliens humans crew human

3	0.777	Star Trek Nemesis
3	0.614	Star Trek
3	0.613	Star Trek: The Motion Picture
3	0.601	Supernova
3	0.594	Galaxy Quest

stone kid dj vega kramer pop smoke play party stab

4	0.424	House Party
4	0.226	Biker Boyz
4	0.214	House Party 2
4	0.178	House Party 3
4	0.158	Stomp the Yard

mrs. boone camp kids cat hayes whale butch tooth star